In [13]:
import findspark
findspark.init()

In [15]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.streaming import *
from pyspark.sql.types import *
from pyspark.sql.window import Window
import os

spark = SparkSession.builder.master("local").appName("TaskStages").getOrCreate()
sc = spark.sparkContext

In [16]:
spark

In [17]:
# sql adaptive query execution adaptive.coalescePartitions.enabled will makr paritions dynamic
sc.setLogLevel("Error")
#spark.conf.set("spark.sql.shuffle.partitions",3)
#spark.conf.get("spark.sql.shuffle.partitions")
#spark.conf.set("spark.sql.adaptive.enabled","false")
#spark.conf.set("spark.sql.adaptive.coalescePartitions.enabled","false")
#spark.conf.set("spark.sql.execution.arrow.enabled",True)
#spark.conf.set("spark.sql.execution.arrow.fallback.enabled",True)

"""
Read the file with timestamp as format. How  many tasks and stages are involved
T1 - read csv
T2 filter
A1- collect
T3 - add column
T4 - filter
A2 - show
"""

#WithoutInferSchema
#headerTrue will read first row and assign column names but type is String for all
#spark job created to read first column
filepath = "file:///C:/Users/venka/PycharmProjects/pythonProject/dataset/"
df = spark.read.option("header",True) \
            .option("inferSchema",True) \
            .option("delimiter",",") \
            .csv(filepath + "IntPersonal_transactions.csv")

df1 = df.filter(df["Card_type"]=="Platinum Card")
df2 = df1.collect()
df3 = df1.withColumn("CollectData",lit("Platinum"))
df4 = df3.filter(df3["Date"]=="1/13/2018")
df4.show()

+-----------+-------------+---------+---------+----------------+------+-----------+
|Customer_No|    Card_type|     Date| Category|Transaction Type|Amount|CollectData|
+-----------+-------------+---------+---------+----------------+------+-----------+
|    1000531|Platinum Card|1/13/2018|Fast Food|           debit| 32.91|   Platinum|
|    1000531|Platinum Card|1/13/2018| Shopping|           debit| 39.05|   Platinum|
+-----------+-------------+---------+---------+----------------+------+-----------+

